In [2]:
# Before that, install spark, add installation bin folder to path
# install jdk , add jdk installation folder to JAVA_HOME env
# install pyspark

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [7]:
from pyspark.sql import types as T
from pyspark.sql import functions as F

## 1. Bring Data into df

In [5]:
df= spark.read.format("csv").option("header","true").load("original.csv")

In [6]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [7]:
df=spark.read.csv("original.csv",header=True)

In [8]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [9]:
df2=spark.read.text("test.txt")

it rads text to df as 1 line per row

In [10]:
df2.show()

+--------------------+
|               value|
+--------------------+
|Notepad++ v8.6.5 ...|
|                    |
| 1. Fix "Replace ...|
| 2. Fix plugins n...|
| 3. Fix Shortcut ...|
| 4. Fix period ba...|
| 5. Fix NUL chara...|
| 6. Remedy losing...|
| 7. Fix URLs are ...|
| 8. Notify user w...|
| 9. Update to sci...|
|10. Support templ...|
|11. Add support f...|
|12. Fix NPPM_RELO...|
|13. Enhance Short...|
|14. Prevent typin...|
|15. Fix possible ...|
|16. Make context ...|
|17. Fix Notepad++...|
|18. Fix python wr...|
+--------------------+
only showing top 20 rows



In [11]:
df.dtypes

[('id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string')]

### define Schema

In [12]:
def calc_num_of_nulls(df_inp):
    total_nulls = df_inp.select([F.sum(F.col(column).isNull().cast("int")).alias(column) for column in df_inp.columns])
    # Sum the counts across all columns to get the total null count
    total_null_count = 0
    for row in total_nulls.collect()[0]:
        total_null_count += row

    return total_null_count

In [13]:
# my_schema = T.StructType(
#     [
#         T.StructField("id",T.IntegerType()),
#         T.StructField("first_name",T.StringType()),
#         T.StructField('last_name', T.StringType()),
#         T.StructField('gender', T.StringType()),
#         T.StructField('city', T.StringType()),
#         T.StructField('job_title', T.StringType()),
#         T.StructField('salary', T.FloatType()),
#         T.StructField('latitude', T.FloatType()),
#         T.StructField('longitude', T.FloatType())
        
#     ]
# )

In [14]:
# TODO : find out how to apply schema on existing df and also rename some columns
#df_new = df.select(*[df[col].cast(my_schema[col].dataType).alias(my_schema.fields[i].name) for i, col in enumerate(df.columns)])

In [15]:
# df = spark.read.csv("original.csv", header=True, schema =my_schema)

In [16]:
# calc_num_of_nulls(df)

1004

In [17]:
# df.show()

+---+----------+----------+------+---------------+--------------------+------+----------+----------+
| id|first_name| last_name|gender|           city|           job_title|salary|  latitude| longitude|
+---+----------+----------+------+---------------+--------------------+------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|  NULL| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|  NULL|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|  NULL| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|  NULL| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|  NULL|      NULL| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|  NULL| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|                NULL|  NULL| 24.879416|11

In [18]:
# TODO : how to push pyspark to raise error/warning when can not use specific type for a column ( and not only fill with nulls)

In [19]:
df_without_schema = spark.read.csv("original.csv", header=True)

In [18]:
my_schema = T.StructType(
    [
        T.StructField("id",T.IntegerType()),
        T.StructField("first_name",T.StringType()),
        T.StructField('last_name', T.StringType()),
        T.StructField('gender', T.StringType()),
        T.StructField('city', T.StringType()),
        T.StructField('job_title', T.StringType()),
        T.StructField('salary', T.StringType()),
        T.StructField('latitude', T.FloatType()),
        T.StructField('longitude', T.FloatType())
        
    ]
)

In [19]:
df = spark.read.csv("original.csv", header=True, schema=my_schema)

In [22]:
calc_num_of_nulls(df)

4

In [23]:
df.dtypes

[('id', 'int'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('city', 'string'),
 ('job_title', 'string'),
 ('salary', 'string'),
 ('latitude', 'float'),
 ('longitude', 'float')]

## 2. Inspect Data

In [24]:
df.columns

['id',
 'first_name',
 'last_name',
 'gender',
 'city',
 'job_title',
 'salary',
 'latitude',
 'longitude']

In [25]:
df.dtypes

[('id', 'int'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('city', 'string'),
 ('job_title', 'string'),
 ('salary', 'string'),
 ('latitude', 'float'),
 ('longitude', 'float')]

In [26]:
df.head(3)

[Row(id=1, first_name='Melinde', last_name='Shilburne', gender='Female', city='Nowa Ruda', job_title='Assistant Professor', salary='$57438.18', latitude=50.57740783691406, longitude=16.49671745300293),
 Row(id=2, first_name='Kimberly', last_name='Von Welden', gender='Female', city='Bulgan', job_title='Programmer II', salary='$62846.60', latitude=48.823158264160156, longitude=103.52182006835938),
 Row(id=3, first_name='Alvera', last_name='Di Boldi', gender='Female', city=None, job_title=None, salary='$57576.52', latitude=39.994747161865234, longitude=116.33977508544922)]

In [27]:
df.sample(fraction=0.01).collect()

[Row(id=277, first_name='Sonny', last_name='Grent', gender='Male', city='Yatsushiro', job_title='Senior Cost Accountant', salary='$85517.70', latitude=35.791404724121094, longitude=140.27920532226562),
 Row(id=538, first_name='Marcellina', last_name='Samarth', gender='Female', city='Cacheu', job_title='Web Developer II', salary='$13772.82', latitude=12.273262977600098, longitude=-16.166915893554688),
 Row(id=685, first_name='Florrie', last_name='Hodgets', gender='Female', city='Viejo Daan Banua', job_title='Editor', salary='$43927.50', latitude=10.903539657592773, longitude=123.05970001220703),
 Row(id=690, first_name='Carroll', last_name='Bolduc', gender='Male', city='Hoorn', job_title='Civil Engineer', salary='$14724.40', latitude=52.647239685058594, longitude=5.05704402923584),
 Row(id=705, first_name='Fredek', last_name='Gibbings', gender='Male', city='Jamaica', job_title='Social Worker', salary='$36978.51', latitude=40.69989013671875, longitude=-73.8076400756836),
 Row(id=754, fir

In [28]:
df.tail(3)

[Row(id=998, first_name='Holly-anne', last_name='Gerbl', gender='Female', city='Guanaja', job_title='Speech Pathologist', salary='$16200.10', latitude=16.482662200927734, longitude=-85.87932586669922),
 Row(id=999, first_name='Marysa', last_name='Purdie', gender='Female', city='Sioah', job_title='Desktop Support Technician', salary='$95912.44', latitude=5.586496353149414, longitude=95.35140228271484),
 Row(id=1000, first_name='Jewell', last_name='Haselden', gender='Female', city='Springfield', job_title='Business Systems Development Analyst', salary='$30556.12', latitude=39.79999923706055, longitude=-89.6500015258789)]

In [29]:
df.first()

Row(id=1, first_name='Melinde', last_name='Shilburne', gender='Female', city='Nowa Ruda', job_title='Assistant Professor', salary='$57438.18', latitude=50.57740783691406, longitude=16.49671745300293)

In [30]:
df.describe().show() ## TODO : required in report

+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+------------------+-----------------+
|summary|               id|first_name|last_name|gender|               city|          job_title|   salary|          latitude|        longitude|
+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+------------------+-----------------+
|  count|             1000|      1000|     1000|  1000|                999|                998|     1000|               999|             1000|
|   mean|            500.5|      NULL|     NULL|  NULL|               NULL|               NULL|     NULL| 25.43151724702484|43.33756460386515|
| stddev|288.8194360957494|      NULL|     NULL|  NULL|               NULL|               NULL|     NULL|24.579082550156635| 69.4206453674681|
|    min|                1|   Abagail|    Abbay|Female|             Abéché|Account Coordinator|$10101.92|         -54.28115|       -123.04196|

In [31]:
df.summary().show()  

+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+------------------+-----------------+
|summary|               id|first_name|last_name|gender|               city|          job_title|   salary|          latitude|        longitude|
+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+------------------+-----------------+
|  count|             1000|      1000|     1000|  1000|                999|                998|     1000|               999|             1000|
|   mean|            500.5|      NULL|     NULL|  NULL|               NULL|               NULL|     NULL| 25.43151724702484|43.33756460386515|
| stddev|288.8194360957494|      NULL|     NULL|  NULL|               NULL|               NULL|     NULL|24.579082550156635| 69.4206453674681|
|    min|                1|   Abagail|    Abbay|Female|             Abéché|Account Coordinator|$10101.92|         -54.28115|       -123.04196|

In [32]:
df.count() # total number of rows  ## TODO : required in report

1000

In [33]:
df.select(df.gender).distinct().count()  # find if a column is category  ## TODO : required in report

2

In [34]:
df.distinct().count() # total number of distinct rows

1000

## Handling Null & duplicates

In [35]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           city|           job_title|   salary|  latitude| longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

### dropping Nulls

In [36]:
df_dropped = df.na.drop()  ## drops any row that contain any na
df_dropped.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           city|           job_title|   salary|  latitude| longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36| 45.190517| 0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Product Manage...|$73697.10| 32.027935| 106.65711|
| 10|      Bran|   Trahear|  Male|       Arbeláez|Mechanical Sys

In [37]:
df.filter(df.job_title.isNull()).show()

+---+----------+---------+------+------+---------+---------+---------+----------+
| id|first_name|last_name|gender|  city|job_title|   salary| latitude| longitude|
+---+----------+---------+------+------+---------+---------+---------+----------+
|  3|    Alvera| Di Boldi|Female|  NULL|     NULL|$57576.52|39.994747|116.339775|
|  7|     Masha|   Divers|Female|Dachun|     NULL|$25090.87|24.879416|118.930115|
+---+----------+---------+------+------+---------+---------+---------+----------+



In [38]:
df.filter(df.job_title.isNotNull()).show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           city|           job_title|   salary|  latitude| longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36| 45.190517| 0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Product Man

### replacing Nulls 

In [20]:
df_clean = df.withColumn("city",F.when(df.city.isNull(),"Unknown").otherwise(df.city)) # It overwrites the original column and keep it in its location!
df_clean.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           city|           job_title|   salary|  latitude| longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|        Unknown|                NULL|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

### drop duplicates

In [ ]:
df.dropDuplicates().show()  # return the df after dropping duplicated rows

In [52]:
df.dropDuplicates(subset=["id"]).show()   # TODO : used to make sure that pk (id) is unique

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           city|           job_title|   salary|  latitude| longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [54]:
duplicates = df.subtract(df.dropDuplicates(subset=["first_name"])) #Show which are the duplicated rows
duplicates.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           city|           job_title|   salary|  latitude| longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|661|    Emmett| Robertson|  Male|        Kuantan|Administrative Of...|$66460.27| 3.9671435| 103.40391|
|373|     Rocky|  McConway|  Male|      Bradenton|     Sales Associate|$12246.73| 27.440405| -82.58284|
|585|    Janith|     Hanse|Female|     Ridderkerk|             Teacher|$59588.41|  51.85302| 4.6175275|
|804|   Hillyer|    Ancell|  Male|           Góra|Occupational Ther...|$37925.57| 51.669945| 16.537653|
|631|     Barbi|   Salmons|Female|        Leticia|        Developer IV|$18561.11|  -11.3827| -75.75604|
|978|  Bancroft| Daughtrey|  Male|         Sobang| Associate Professor|$94147.65|-6.6111073| 105.77033|
|896|     Luigi|  Cabrales|  Male|     Koygorodok|              

## 3. Selecting and Filtering

### Selecting col and applying expressions

In [58]:
df.select(df.first_name,df.last_name).show()

+----------+----------+
|first_name| last_name|
+----------+----------+
|   Melinde| Shilburne|
|  Kimberly|Von Welden|
|    Alvera|  Di Boldi|
|   Shannon| O'Griffin|
|  Sherwood|   Macieja|
|     Maris|      Folk|
|     Masha|    Divers|
|   Goddart|     Flear|
|      Roth|O'Cannavan|
|      Bran|   Trahear|
|    Kylynn|   Lockart|
|       Rey|    Meharg|
|      Kerr|    Braden|
|    Mickie| Whanstall|
|    Kaspar|     Pally|
|    Norbie|    Gwyllt|
|    Claude|    Briant|
|     Thain|    Habbon|
|  Tiffanie|  Pattison|
|    Ettore|  Gerriets|
+----------+----------+
only showing top 20 rows



In [66]:
df.select(F.concat(df.first_name,F.lit(" "),df.last_name).alias("full_name")).show()

+-------------------+
|          full_name|
+-------------------+
|  Melinde Shilburne|
|Kimberly Von Welden|
|    Alvera Di Boldi|
|  Shannon O'Griffin|
|   Sherwood Macieja|
|         Maris Folk|
|       Masha Divers|
|      Goddart Flear|
|    Roth O'Cannavan|
|       Bran Trahear|
|     Kylynn Lockart|
|         Rey Meharg|
|        Kerr Braden|
|   Mickie Whanstall|
|       Kaspar Pally|
|      Norbie Gwyllt|
|      Claude Briant|
|       Thain Habbon|
|  Tiffanie Pattison|
|    Ettore Gerriets|
+-------------------+
only showing top 20 rows



In [75]:
df.select(F.concat(df.first_name , F.repeat(df.first_name, 2)).alias("concatenated_name")).show()

+--------------------+
|   concatenated_name|
+--------------------+
|MelindeMelindeMel...|
|KimberlyKimberlyK...|
|  AlveraAlveraAlvera|
|ShannonShannonSha...|
|SherwoodSherwoodS...|
|     MarisMarisMaris|
|     MashaMashaMasha|
|GoddartGoddartGod...|
|        RothRothRoth|
|        BranBranBran|
|  KylynnKylynnKylynn|
|           ReyReyRey|
|        KerrKerrKerr|
|  MickieMickieMickie|
|  KasparKasparKaspar|
|  NorbieNorbieNorbie|
|  ClaudeClaudeClaude|
|     ThainThainThain|
|TiffanieTiffanieT...|
|  EttoreEttoreEttore|
+--------------------+
only showing top 20 rows



In [63]:
df.select((df.latitude + df.longitude).alias("sum")).show()

+----------+
|       sum|
+----------+
|  67.07413|
| 152.34497|
| 156.33452|
| 82.634735|
|      NULL|
|  47.26211|
| 143.80952|
|  45.93283|
| 138.68504|
| -70.14322|
| -85.73333|
| 156.10399|
|  69.19235|
|-30.475197|
| 63.283863|
| 49.392376|
|  70.07203|
| 29.986412|
| 105.74542|
|  32.35469|
+----------+
only showing top 20 rows



In [77]:
df.select(df.first_name.alias("f_name")).show()

+--------+
|  f_name|
+--------+
| Melinde|
|Kimberly|
|  Alvera|
| Shannon|
|Sherwood|
|   Maris|
|   Masha|
| Goddart|
|    Roth|
|    Bran|
|  Kylynn|
|     Rey|
|    Kerr|
|  Mickie|
|  Kaspar|
|  Norbie|
|  Claude|
|   Thain|
|Tiffanie|
|  Ettore|
+--------+
only showing top 20 rows



In [86]:
import math
df.select(math.sqrt(df.longitude)).show()

TypeError: must be real number, not Column

In [80]:
df.selectExpr("sqrt(longitude)", "log(latitude)").show()

+------------------+------------------+
|   SQRT(longitude)|LOG(E(), latitude)|
+------------------+------------------+
|4.0616151286160695|3.9235049911526376|
| 10.17456731602673| 3.888204754887127|
|10.786091742862622|3.6887481245372156|
| 6.174950714532408|3.7955953092764223|
| 6.135877642668633|              NULL|
|               NaN|3.9783089898745434|
|10.905508458852056| 3.214040775315487|
|0.8615755171375312|3.8108872733584587|
|  10.3274930221028| 3.466608491610733|
|               NaN|1.4522676687468663|
|               NaN|              NULL|
|10.813492040382787| 3.667971869348032|
| 4.401951201165389| 3.908319791631844|
|               NaN|3.7400828606573744|
|3.6730821289136224|3.9078609901572245|
| 2.428539041495548|3.7726362291806046|
| 4.162675064575339| 3.965453194259342|
|               NaN| 3.655742621194683|
|10.652167917163586|              NULL|
|               NaN|3.7066507315664965|
+------------------+------------------+
only showing top 20 rows



In [88]:
df.select(F.sqrt(df.longitude).alias("square_root"),F.log(df.latitude)).show() ## prefered

+------------------+------------------+
|       square_root|      ln(latitude)|
+------------------+------------------+
|4.0616151286160695|3.9235049911526376|
| 10.17456731602673| 3.888204754887127|
|10.786091742862622|3.6887481245372156|
| 6.174950714532408|3.7955953092764223|
| 6.135877642668633|              NULL|
|               NaN|3.9783089898745434|
|10.905508458852056| 3.214040775315487|
|0.8615755171375312|3.8108872733584587|
|  10.3274930221028| 3.466608491610733|
|               NaN|1.4522676687468663|
|               NaN|              NULL|
|10.813492040382787| 3.667971869348032|
| 4.401951201165389| 3.908319791631844|
|               NaN|3.7400828606573744|
|3.6730821289136224|3.9078609901572245|
| 2.428539041495548|3.7726362291806046|
| 4.162675064575339| 3.965453194259342|
|               NaN| 3.655742621194683|
|10.652167917163586|              NULL|
|               NaN|3.7066507315664965|
+------------------+------------------+
only showing top 20 rows



In [84]:
df.select(F.expr("sqrt(longitude)"), F.expr("log(latitude)")).show()

+------------------+------------------+
|   SQRT(longitude)|LOG(E(), latitude)|
+------------------+------------------+
|4.0616151286160695|3.9235049911526376|
| 10.17456731602673| 3.888204754887127|
|10.786091742862622|3.6887481245372156|
| 6.174950714532408|3.7955953092764223|
| 6.135877642668633|              NULL|
|               NaN|3.9783089898745434|
|10.905508458852056| 3.214040775315487|
|0.8615755171375312|3.8108872733584587|
|  10.3274930221028| 3.466608491610733|
|               NaN|1.4522676687468663|
|               NaN|              NULL|
|10.813492040382787| 3.667971869348032|
| 4.401951201165389| 3.908319791631844|
|               NaN|3.7400828606573744|
|3.6730821289136224|3.9078609901572245|
| 2.428539041495548|3.7726362291806046|
| 4.162675064575339| 3.965453194259342|
|               NaN| 3.655742621194683|
|10.652167917163586|              NULL|
|               NaN|3.7066507315664965|
+------------------+------------------+
only showing top 20 rows



In [ ]:
df.select("*", df.first_name.substr(1,5).alias("sub_f_name")).show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+
| id|first_name| last_name|gender|           city|           job_title|   salary|  latitude| longitude|sub_f_name|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|     Melin|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|     Kimbe|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|$57576.52| 39.994747|116.339775|     Alver|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|     Shann|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.648994|     Sherw|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16

### Selecting rows
In PySpark, there isn't a built-in function to directly show a specific row by its index

Note: actions pushes pyspark to cal the action @executers and bring ruslt to the driver ( in this case first 3 rows will be copied to the driver)
So in Big data avoid using actions whenever possible.

In [160]:
df.take(3)[-1]  ## showing 3rd row

Row(id=3, first_name='Alvera', last_name='Di Boldi', gender='Female', city=None, job_title=None, salary=57576.51953125, latitude=39.994747161865234, longitude=116.33977508544922)

### Renaming

In [92]:
df.withColumnRenamed("first_name","f_name").show()  # columns name has to string!

+---+--------+----------+------+---------------+--------------------+---------+----------+----------+
| id|  f_name| last_name|gender|           city|           job_title|   salary|  latitude| longitude|
+---+--------+----------+------+---------------+--------------------+---------+----------+----------+
|  1| Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|  Alvera|  Di Boldi|Female|           NULL|                NULL|$57576.52| 39.994747|116.339775|
|  4| Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.648994|
|  6|   Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|   Masha|    Divers|Female|         Dachun|                NULL|$25090.87| 24

### Filtering

In [40]:
df.filter((df.gender == "Male") & (df.city != "Pedra")).show()

+---+----------+-----------+------+--------------------+--------------------+---------+----------+----------+
| id|first_name|  last_name|gender|                city|           job_title|   salary|  latitude| longitude|
+---+----------+-----------+------+--------------------+--------------------+---------+----------+----------+
|  4|   Shannon|  O'Griffin|  Male|       Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|    Macieja|  Male|           Mytishchi|            VP Sales|$63863.09|      NULL| 37.648994|
|  8|   Goddart|      Flear|  Male|           Trélissac|Desktop Support T...|$46116.36| 45.190517| 0.7423124|
|  9|      Roth| O'Cannavan|  Male|              Heitan|VP Product Manage...|$73697.10| 32.027935| 106.65711|
| 10|      Bran|    Trahear|  Male|            Arbeláez|Mechanical System...|$68098.42|  4.272793|-74.416016|
| 13|      Kerr|     Braden|  Male|           Sułkowice|Compensation Analyst|$33432.99|  49.81518| 19.377174|
| 14|    M

with function.like()  , 2 wildcards can be used
% --> match 0 to n number of chars
_ --> match exact 1 char

In [99]:
df.filter(df.first_name.like("L%a")).show()

+---+----------+---------+------+------------------+--------------------+---------+----------+----------+
| id|first_name|last_name|gender|              city|           job_title|   salary|  latitude| longitude|
+---+----------+---------+------+------------------+--------------------+---------+----------+----------+
| 26|   Leandra|   Anfrey|Female|           Isfahan|VP Product Manage...|$30201.32|  32.65463| 51.667984|
| 59|      Lusa|    Kowal|Female|          Registro|   Financial Advisor|$43176.23|-24.500229| -47.84481|
|139|  Laetitia|  Gasking|Female|           Cuogang|    Registered Nurse|$10908.22|  32.82794| 13.381834|
|179|   Laverna|Yuryichev|Female|        Lyaskovets| Marketing Assistant|$95106.34| 43.068382| 25.742846|
|240|   Lorenza| Brainsby|Female|            Ayorou|     Cost Accountant|$42273.42| 14.732815| 0.9104334|
|451|     Liesa|  Forsyde|Female|          Svorónos|               Nurse|$90634.78| 40.268356| 22.462448|
|520|  Lauretta| Seeborne|Female|Fontenay-sous

In [94]:
df.filter(df.first_name.like("%vera")).show()

+---+----------+---------+------+----+---------+---------+---------+----------+
| id|first_name|last_name|gender|city|job_title|   salary| latitude| longitude|
+---+----------+---------+------+----+---------+---------+---------+----------+
|  3|    Alvera| Di Boldi|Female|NULL|     NULL|$57576.52|39.994747|116.339775|
+---+----------+---------+------+----+---------+---------+---------+----------+



In [102]:
df.filter(df.first_name.endswith("tta") & df.last_name.startswith("S")).show()

+---+----------+---------+------+------------------+------------+---------+---------+---------+
| id|first_name|last_name|gender|              city|   job_title|   salary| latitude|longitude|
+---+----------+---------+------+------------------+------------+---------+---------+---------+
|520|  Lauretta| Seeborne|Female|Fontenay-sous-Bois|Developer IV|$23725.49|48.849396|2.4751086|
|632| Georgetta| Scothron|Female|             Wujia|   Recruiter|$81269.31| 43.31536|129.76788|
+---+----------+---------+------+------------------+------------+---------+---------+---------+



In [104]:
df.filter(df.id.between(1,5)).show()

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
| id|first_name| last_name|gender|         city|           job_title|   salary| latitude| longitude|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|         NULL|                NULL|$57576.52|39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|     NULL| 37.648994|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+



In [106]:
df.filter(df.first_name.isin("Aldin","Valma")).show()

+---+----------+-------------+------+-----------+----------------+---------+----------+---------+
| id|first_name|    last_name|gender|       city|       job_title|   salary|  latitude|longitude|
+---+----------+-------------+------+-----------+----------------+---------+----------+---------+
|569|     Valma|      Bratton|Female|  Kurayoshi|Web Developer II|$32665.89| 35.449905|133.76134|
|901|     Aldin|Matuszkiewicz|  Male|East London|        Operator|$41468.83|-32.954933|27.931913|
+---+----------+-------------+------+-----------+----------------+---------+----------+---------+



In [114]:
df.filter(df.first_name.isin("Aldin","Valma") & df.city.like("%don")).show()

+---+----------+-------------+------+-----------+---------+---------+----------+---------+
| id|first_name|    last_name|gender|       city|job_title|   salary|  latitude|longitude|
+---+----------+-------------+------+-----------+---------+---------+----------+---------+
|901|     Aldin|Matuszkiewicz|  Male|East London| Operator|$41468.83|-32.954933|27.931913|
+---+----------+-------------+------+-----------+---------+---------+----------+---------+



## 4. Running SQL

In [116]:
df.createOrReplaceTempView(" df_as_table")

In [120]:
spark.sql("select * from df_as_table limit 1").show()

+---+----------+---------+------+---------+-------------------+---------+---------+---------+
| id|first_name|last_name|gender|     city|          job_title|   salary| latitude|longitude|
+---+----------+---------+------+---------+-------------------+---------+---------+---------+
|  1|   Melinde|Shilburne|Female|Nowa Ruda|Assistant Professor|$57438.18|50.577408|16.496717|
+---+----------+---------+------+---------+-------------------+---------+---------+---------+



In [122]:
spark.sql("select concat(first_name , ' ' , last_name) as full_name from df_as_table where gender='Female'").show()

+-------------------+
|          full_name|
+-------------------+
|  Melinde Shilburne|
|Kimberly Von Welden|
|    Alvera Di Boldi|
|         Maris Folk|
|       Masha Divers|
|     Kylynn Lockart|
|         Rey Meharg|
|      Claude Briant|
|  Tiffanie Pattison|
|    Lurleen Janczak|
|      Nichol Holtum|
|       Shaun Bridle|
|     Leandra Anfrey|
|    Jaquelyn Hazard|
|  Prudence Honacker|
|       Cherey Liger|
|          Neda Krop|
|    Barbi Fattorini|
|   Lonnie Townshend|
|    Valida Salzberg|
+-------------------+
only showing top 20 rows



## 5. Adding Calculate columns

In [129]:
df =df.withColumn("salary",df.salary.substr(2,1000).cast("float"))

In [130]:
df.show()

+---+----------+----------+------+---------------+--------------------+--------+----------+----------+
| id|first_name| last_name|gender|           city|           job_title|  salary|  latitude| longitude|
+---+----------+----------+------+---------------+--------------------+--------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II| 62846.6|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|63863.09|      NULL| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|                NULL|25

In [133]:
df.withColumn("monthly_salary", F.round(df.salary/12,2)).show()

+---+----------+----------+------+---------------+--------------------+--------+----------+----------+--------------+
| id|first_name| last_name|gender|           city|           job_title|  salary|  latitude| longitude|monthly_salary|
+---+----------+----------+------+---------------+--------------------+--------+----------+----------+--------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|57438.18| 50.577408| 16.496717|       4786.51|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II| 62846.6|  48.82316| 103.52182|       5237.22|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|57576.52| 39.994747|116.339775|       4798.04|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|61489.23| 44.504723| 38.130016|        5124.1|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|63863.09|      NULL| 37.648994|       5321.92|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      

In [17]:
df.withColumn("are_female", F.when(df.gender == "Female","yes").otherwise("no")).show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+
| id|first_name| last_name|gender|           city|           job_title|   salary|  latitude| longitude|are_female|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|       yes|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|       yes|
|  3|    Alvera|  Di Boldi|Female|           NULL|                NULL|$57576.52| 39.994747|116.339775|       yes|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|        no|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      NULL| 37.648994|        no|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16

## 6. GroupBy and Aggregate

### GroupBy

In [137]:
df.groupBy(df.gender).agg(F.sum(df.salary).alias("sum")).show()

+------+--------------------+
|gender|                 sum|
+------+--------------------+
|Female|2.7364519950195312E7|
|  Male|2.8123435678710938E7|
+------+--------------------+



In [16]:
df.groupBy(df.city,df.gender).agg(F.sum(df.salary)).orderBy(df.city.desc()).show()

+-------------------+------+-----------+
|               city|gender|sum(salary)|
+-------------------+------+-----------+
|‘Aşīrah al Qiblīyah|  Male|       NULL|
|             Ḩajjah|  Male|       NULL|
|             Žrnovo|  Male|       NULL|
| Šmartno pri Litiji|  Male|       NULL|
|            Šluknov|Female|       NULL|
|               Ōami|Female|       NULL|
|         Đông Thành|  Male|       NULL|
|            Ādilpur|  Male|       NULL|
|             Öjebyn|  Male|       NULL|
|             Íquira|Female|       NULL|
|               Évry|  Male|       NULL|
|           Zongzhai|  Male|       NULL|
|             Zinder|  Male|       NULL|
|              Zhuxi|  Male|       NULL|
|              Zhuqi|  Male|       NULL|
|           Zhujiang|Female|       NULL|
|         Zhuanqukou|  Male|       NULL|
|            Zhosaly|  Male|       NULL|
|          Zhongying|  Male|       NULL|
|         Zhongcheng|  Male|       NULL|
+-------------------+------+-----------+
only showing top

In [162]:
df.select(df.salary).describe().show()

+-------+-----------------+
|summary|           salary|
+-------+-----------------+
|  count|             1000|
|   mean|55487.95562890625|
| stddev|25855.22985831031|
|    min|         10101.92|
|    max|         99948.28|
+-------+-----------------+



In [168]:
# groupBy a column range
(df.withColumn("salary_range", 
                            F.when((df.salary >= 10000) & (df.salary <= 40000), "10k-40k")
                            .otherwise(
                                F.when((df.salary > 40000) & (df.salary <= 70000), "40k-70k")
                                .otherwise(
                                    F.when((df.salary > 70000) & (df.salary <= 100000), "70k-100k")
                                    .otherwise("Other"))))
               .groupBy("salary_range")
               .agg(F.count("*").alias("count"))).show()

+------------+-----+
|salary_range|count|
+------------+-----+
|    70k-100k|  334|
|     10k-40k|  327|
|     40k-70k|  339|
+------------+-----+



### Aggregate
options : sum,avg,min,max,count,countDistinct,first,last,collect_list,collect_set,sumDistinct

In [176]:
(df.groupBy(df.gender).agg(F.round(F.avg(df.salary),2).alias("salary_avg"),
                          F.count(df.first_name).alias("count")
                          )
  .show())

+------+----------+-----+
|gender|salary_avg|count|
+------+----------+-----+
|Female|  55618.94|  492|
|  Male|  55361.09|  508|
+------+----------+-----+



In [179]:
(df.groupBy(df.gender).agg((F.round(F.sum(df.salary)/F.count(df.first_name),2)).alias("salary_avg"))
  .show())

+------+----------+
|gender|salary_avg|
+------+----------+
|Female|  55618.94|
|  Male|  55361.09|
+------+----------+



In [181]:
(df.groupBy(df.gender).agg({"salary": "sum", "city": "min"})
  .show())

+------+---------+--------------------+
|gender|min(city)|         sum(salary)|
+------+---------+--------------------+
|Female|   Abéché|2.7364519950195312E7|
|  Male|Ad Dimnah|2.8123435678710938E7|
+------+---------+--------------------+



### sorting

In [13]:
df.groupBy(df.city,df.gender).agg(F.sum(df.salary).alias("sum")).orderBy(df.city.desc()).show()

+-------------------+------+----+
|               city|gender| sum|
+-------------------+------+----+
|‘Aşīrah al Qiblīyah|  Male|NULL|
|             Ḩajjah|  Male|NULL|
|             Žrnovo|  Male|NULL|
| Šmartno pri Litiji|  Male|NULL|
|            Šluknov|Female|NULL|
|               Ōami|Female|NULL|
|         Đông Thành|  Male|NULL|
|            Ādilpur|  Male|NULL|
|             Öjebyn|  Male|NULL|
|             Íquira|Female|NULL|
|               Évry|  Male|NULL|
|           Zongzhai|  Male|NULL|
|             Zinder|  Male|NULL|
|              Zhuxi|  Male|NULL|
|              Zhuqi|  Male|NULL|
|           Zhujiang|Female|NULL|
|         Zhuanqukou|  Male|NULL|
|            Zhosaly|  Male|NULL|
|          Zhongying|  Male|NULL|
|         Zhongcheng|  Male|NULL|
+-------------------+------+----+
only showing top 20 rows



In [21]:
df.groupBy(df.city,df.gender).agg(F.sum(df.salary).alias("sum")).orderBy(F.col("sum").desc()).show()

+------------+------+----+
|        city|gender| sum|
+------------+------+----+
|   Kaustinen|Female|NULL|
|  Sungai Iyu|Female|NULL|
|      Toledo|  Male|NULL|
|    Slobodka|Female|NULL|
|  Kharagauli|  Male|NULL|
|    Rockford|  Male|NULL|
|    Floresta|Female|NULL|
|      Gaoshi|  Male|NULL|
|  Tsaghkunk’|Female|NULL|
|   Balahovit|  Male|NULL|
|        Idkū|  Male|NULL|
| Maple Ridge|  Male|NULL|
|    Mrganush|Female|NULL|
| Tasikmalaya|Female|NULL|
|      Tabant|Female|NULL|
|Casal Galego|  Male|NULL|
|     Dahuang|  Male|NULL|
|        Same|Female|NULL|
|   Mieścisko|Female|NULL|
|      Burgau|  Male|NULL|
+------------+------+----+
only showing top 20 rows



## 7. Write DF to Files

In [10]:
df.write.csv("out.csv")

Py4JJavaError: An error occurred while calling o40.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:6

## x. String handling
df.col....  --> like, rlike, contains, endswith, startswith, substr
F..... --> concat, concat_ws, split,substring,length,trim,ltrim,rtrim,lower,upper,lpad,rpad,regex_extract,regex_replace,translate,initcap,...

## xx. Integer Handling
df.col.... --> between,isin,<, <=, >, >=,isNaN,equalTo,remainder,shiftLeft, shiftRight
bitwise op & (AND), | (OR), ~ (NOT)
F..... --> abs, add,subtract, multiply, divide, mod, pow, bitwiseAND, bitwiseOR, bitwiseXOR, bitwiseNOT, shiftLeft, shiftRight, greatest, least, bin, rand, isnan..